In [79]:
# # ChromaDB Admin Interface
# Админские функции для управления векторной базой

In [80]:
import os
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd

from dotenv import load_dotenv

from chromadb import Client, Settings
from chromadb.utils import embedding_functions

from sentence_transformers import SentenceTransformer

from IPython.display import display
import ipywidgets as widgets

In [81]:
# Настройки
DB_PATH = "./db"
COLLECTION_NAME = "docs"
EMBEDDING_MODEL = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
EMBEDDING_DIM = 768

In [82]:
# Загрузка конфигурации
load_dotenv()

# Модель для эмбеддингов
embedder = SentenceTransformer(EMBEDDING_MODEL)
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=EMBEDDING_MODEL)

# Функция для нормализации эмбеддингов
def normalize_embedding(embedding):
    """Normalize embedding to unit length (L2 norm = 1)."""
    embedding = np.array(embedding)
    norm = np.linalg.norm(embedding)
    if norm > 0:
        return (embedding / norm).tolist()
    return embedding.tolist()

# Инициализация ChromaDB
db = Client(settings=Settings(
    persist_directory="./db",
    allow_reset=True,
    is_persistent=True  # Автосохранение включено
))
collection = db.get_or_create_collection(
    name="docs",
    embedding_function=embedding_func
)

In [83]:
# Проверка подключения
print(f"Доступные коллекции: {db.list_collections()}")
print(f"Используемая коллекция: {collection.name}")

Доступные коллекции: [Collection(name=docs)]
Используемая коллекция: docs


In [84]:
# Основные функции администрирования
def show_collection_stats():
    """Показать статистику коллекции"""
    count = collection.count()
    stats = {
        "Количество документов": count,
        "Размерность эмбеддингов": len(collection.peek()["embeddings"][0]) if count > 0 else 0,
        "Уникальных пользователей": len(set(m.get("user", "unknown") for m in collection.get()["metadatas"])) if count > 0 else 0,
        "Средняя длина документа": np.mean([len(doc) for doc in collection.get()["documents"]]) if count > 0 else 0
    }
    print("\n".join(f"{k}: {v:.2f}" if isinstance(v, float) else f"{k}: {v}" for k, v in stats.items()))

def search_documents(query: str, n_results: int = 5, similarity_threshold: float = 0.5):
    """Семантический поиск с фильтрацией"""
    results = collection.query(
        query_texts=[query],
        n_results=n_results,
        include=["documents", "metadatas", "distances"]
    )
    
    filtered = [
        (doc, meta, 1-dist, id)
        for doc, meta, dist, id in zip(
            results["documents"][0],
            results["metadatas"][0],
            results["distances"][0],
            results["ids"][0]
        )
        if (1 - dist) >= similarity_threshold
    ]
    
    return pd.DataFrame(filtered, columns=["Текст", "Метаданные", "Схожесть", "ID"])

def export_to_dataframe():
    """Экспорт всей коллекции в DataFrame с эмбеддингами"""
    data = collection.get(include=["documents", "metadatas", "embeddings"])
    
    # Преобразуем эмбеддинги в список строк для Pandas
    embeddings = [emb for emb in data["embeddings"]] if (data["embeddings"] is not None and len(data["embeddings"]) > 0) else [None]*len(data["ids"])
    
    return pd.DataFrame({
        "ids": data["ids"],
        "documents": data["documents"],
        "metadatas": data["metadatas"],
        "embeddings": embeddings
    })

def add_document(text: str, metadata: dict = {}):
    """Добавить документ в коллекцию"""
    embedding = normalize_embedding(embedder.encode(text).tolist())
    
    if 'source' not in metadata:
        metadata['source'] = 'Admin_Notebook'
    if 'timestamp' not in metadata:
        metadata['timestamp'] = datetime.now().isoformat()
    if 'user' not in metadata:
        metadata['user'] = 'Admin'

    user = metadata['user']

    doc_id = f"doc_{user}_{datetime.now().timestamp()}"

    collection.add(
        documents=[text],
        embeddings=[embedding],
        metadatas=[metadata],
        ids=[doc_id]
    )
    print(f"Документ {doc_id} добавлен")

def delete_document(doc_id: str):
    """Удалить документ по ID"""
    if not collection.get(ids=[doc_id])["ids"]:
        print(f"Документ {doc_id} не найден")
        return
    collection.delete(ids=[doc_id])
    print(f"Документ {doc_id} удален")

def update_document(doc_id: str, new_text: str, new_metadata: dict = {}):
    """Обновить документ"""
    if not collection.get(ids=[doc_id])["ids"]:
        print(f"Документ {doc_id} не найден")
        return
    new_embedding = normalize_embedding(embedder.encode(new_text).tolist())
    
    if 'source' not in new_metadata:
        new_metadata['source'] = 'Admin_Notebook'
    if 'timestamp' not in new_metadata:
        new_metadata['timestamp'] = datetime.now().isoformat()
    if 'user' not in new_metadata:
        new_metadata['user'] = 'Admin'

    collection.update(
        ids=[doc_id],
        documents=[new_text],
        embeddings=[new_embedding],
        metadatas=[new_metadata]
    )
    print(f"Документ {doc_id} обновлен")

def clean_collection():
    """Очистка коллекции с подтверждением"""
    if collection.count() == 0:
        print("Коллекция уже пуста!")
        return
    print(f"Будет удалено {collection.count()} документов")
    while True:
        response = input("Подтвердите очистку (y/n): ").lower().strip()
        if response in ['y', 'n']:
            break
        print("Пожалуйста, введите 'y' или 'n'")
    if response == 'y':
        collection.delete(ids=collection.get()["ids"])
        print("Коллекция очищена")
    else:
        print("Отменено")

def backup_collection():
    """Создание резервной копии"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    backup_dir = "./reserve_copies"
    pathlib.Path(backup_dir).mkdir(exist_ok=True)
    backup_file = f"{backup_dir}/backup_{timestamp}.parquet"
    df = export_to_dataframe()
    df.to_parquet(backup_file)
    print(f"Резервная копия сохранена в {backup_file}")

def restore_from_backup(backup_file: str):
    """Восстановление коллекции из резервной копии"""
    if not os.path.exists(backup_file):
        print(f"Файл {backup_file} не найден")
        return
    df = pd.read_parquet(backup_file)
    if df.empty:
        print("Резервная копия пуста")
        return
    collection.delete(ids=collection.get()["ids"])  # Очистка текущей коллекции
    collection.add(
        ids=df["ids"].tolist(),
        documents=df["documents"].tolist(),
        embeddings=df["embeddings"].tolist(),
        metadatas=df["metadatas"].tolist()
    )
    print(f"Коллекция восстановлена из {backup_file}")

In [85]:
# можно сделать класс
class ChromaAdmin:
    def __init__(self, db_path, collection_name):
        self.client = Client(settings=Settings(
            persist_directory=db_path,
            is_persistent=True
        ))
        self.collection = self.client.get_or_create_collection(collection_name)
        self.embedder = SentenceTransformer(EMBEDDING_MODEL)

In [86]:
# Показать статистику
show_collection_stats()

Количество документов: 6
Размерность эмбеддингов: 768
Уникальных пользователей: 2
Средняя длина документа: 35.17


In [87]:
# Поиск документов
search_results = search_documents("Лучший работник", n_results = 10, similarity_threshold = 0.0)
search_results

,Текст,Метаданные,Схожесть,ID
0,Наш лучший работник месяца - Иван,"{'user': 'alexey511', 'timestamp': '2025-05-23...",0.607899,doc_204015369_1748030726.22037
1,Зарплаты в нашей компании выше рыночной,"{'source': 'Admin_Notebook', 'timestamp': '202...",0.495668,doc_6
2,Цель нашей компании - двигать прогресс вперед,"{'user': 'alexey511', 'source': 'telegram', 't...",0.440849,doc_204015369_1748030719.039531
3,"Наша компания называется ""Вперед""","{'user': 'alexey511', 'source': 'telegram', 't...",0.413332,doc_204015369_1748030744.048663
4,Нашей компании 10 лет,"{'timestamp': '2025-05-23T23:05:12.532886', 'u...",0.411033,doc_204015369_1748030712.498886
5,В нашей компании работает 35 сотрудников,"{'user': 'alexey511', 'timestamp': '2025-05-23...",0.340675,doc_204015369_1748030702.155966


In [88]:
full_data = export_to_dataframe()
full_data

,ids,documents,metadatas,embeddings
0,doc_204015369_1748030702.155966,В нашей компании работает 35 сотрудников,"{'user': 'alexey511', 'timestamp': '2025-05-23...","[-0.02059052884578705, -0.002424149541184306, ..."
1,doc_204015369_1748030712.498886,Нашей компании 10 лет,"{'user': 'alexey511', 'timestamp': '2025-05-23...","[0.03227408975362778, 0.057025112211704254, -0..."
2,doc_204015369_1748030719.039531,Цель нашей компании - двигать прогресс вперед,"{'user': 'alexey511', 'source': 'telegram', 't...","[0.03854742646217346, 0.07083165645599365, -0...."
3,doc_204015369_1748030726.22037,Наш лучший работник месяца - Иван,"{'user': 'alexey511', 'source': 'telegram', 't...","[0.025553634390234947, -0.014779889956116676, ..."
4,doc_204015369_1748030744.048663,"Наша компания называется ""Вперед""","{'source': 'telegram', 'user': 'alexey511', 't...","[0.038271598517894745, 0.028600120916962624, -..."
5,doc_6,Зарплаты в нашей компании выше рыночной,"{'source': 'Admin_Notebook', 'timestamp': '202...","[-0.0335618332028389, 0.041996922343969345, -0..."


In [89]:
text_to_add = "Ты модель для помощи связи бизнеса и клиентов"
add_document(text_to_add)

Документ doc_Admin_1748044589.869966 добавлен


In [90]:
#clean_collection()